# Semantic text search using embeddings

We can search through all our reviews semantically in a very different manner, and at a very low cost, by embedding our search query and then finding similar reviews. The dataset is created in the GetEmbeddingsFromDataset Notebook.

In [2]:
#r "nuget: CsvHelper, 30.0.1"
#load "Csv.fs"
#load "Review.fs"

open Csv
open Review

let dataFilePath = "data/fine_food_reviews_with_embeddings_1k.csv"
let df = readCsvWithMap<AugmentedReview, AugmentedReviewMap> dataFilePath

Installed Packages CsvHelper, 30.0.1

## 1. Read the data
Let's use some of our CSV helper functions to read the data. I am using CsvHelper under the hood, but you can extract this any way you would like.

In [3]:
#r "nuget: System.Numerics.Tensors, 8.0.0-rc.2.23479.6"
#load "OpenAi.fs"

open OpenAi.EmbeddingsUtils

let searchReviews (reviews: AugmentedReview seq) (pprint0: bool option) (n0: int option) (productDescription: string) = 
    let n = defaultArg n0 3
    let pprint = defaultArg pprint0 true
    let productEmbedding = 
        getEmbeddings productDescription "text-embedding-ada-002"
        |> (function
            | Ok embeddings -> embeddings[0].Embedding
            | _ -> [||]) 
        
    let results =
        reviews
        |> Seq.sortByDescending (fun review -> getCosineSimilarity review.Embedding productEmbedding)
        |> Seq.take n

    if pprint then
        for r in results do
            printfn $"{r.Combined}"
            printfn ""
        results
    else
        results
    
let searchTopNReviews = searchReviews df (Some true)
let searchTop3Reviews = searchTopNReviews (Some 3)
let searchTopReview = searchTopNReviews (Some 1)
let searchTop2Reviews = searchTopNReviews (Some 2)

searchTop3Reviews "delicious beans" |> ignore


Installed Packages System.Numerics.Tensors, 8.0.0-rc.2.23479.6

Title: Good Buy; Content: I liked the beans. They were vacuum sealed, plump and moist. Would recommend them for any use. I personally split and stuck them in some vodka to make vanilla extract. Yum!

Title: Jamaican Blue beans; Content: Excellent coffee bean for roasting. Our family just purchased another 5 pounds for more roasting. Plenty of flavor and mild on acidity when roasted to a dark brown bean and before any oil appears on the bean itself (455F @ 17 minutes).

Title: Delicious!; Content: I enjoy this white beans seasoning, it gives a rich flavor to the beans I just love it, my mother in law didn't know about this Zatarain's brand and now she is traying different seasoning and she likes it very much.<br />Thank you Amazon for having it because now I can't find it in stores, I like to have this 12 boxes because I can made it whenever my family want it.



In [4]:
searchTop3Reviews "whole wheat pasta" |> ignore

Title: Tasty and Quick Pasta; Content: Barilla Whole Grain Fusilli with Vegetable Marinara is tasty and has an excellent chunky vegetable marinara.  I just wish there was more of it.  If you aren't starving or on a diet, the 9oz serving is enough for lunch although you might want to add a piece of fruit to feel full.  The whole grain fusilli cooked to al dente tenderness following the instructions and the chunky marinara sauce is so good that I wished there was more of it.  Rarely do I eat sauce alone but this sauce is good enough to.

Title: sooo good; Content: tastes so good. Worth the money. My boyfriend hates wheat pasta and LOVES this. cooks fast tastes great.I love this brand and started buying more of their pastas. Bulk is best.

Title: Handy; Content: Love the idea of ready in a minute pasta and for that alone this product gets praise.  The pasta is whole grain so that's a big plus and it actually comes out al dente.  The vegetable marinara sauce is tasty, but has a high salt c

In [5]:
searchTopReview "bad delivery" |> ignore

Title: great product, poor delivery; Content: The coffee is excellent and I am a repeat buyer.  Problem this time was with the UPS delivery.  They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box.  we lost half of the cups.  Thius is the third time I have written about this matter to Amazon with no results.  Hopefully someone will respond to me.<br /><br />fred santaniello



In [6]:
searchTopReview "spoilt" |> ignore

Title: Extremely dissapointed; Content: Hi,<br />I am very disappointed with the past shipment I received of the ONE coconut water. 3 of the boxes were leaking and the coconut water was spoiled.<br /><br />Thanks.<br /><br />Laks



In [7]:
searchTop2Reviews "pet food" |> ignore

Title: Good food; Content: The only dry food my queen cat will eat. Helps prevent hair balls. Good packaging. Arrives promptly. Recommended by a friend who sells pet food.

Title: The cats like it; Content: My 7 cats like this food but it is a little yucky for the human. Pieces of mackerel swimming in a dark broth. It is billed as a "complete" food and contains carrots, peas and pasta. Why, I don't know, since these are not foods that a cat would eat in the wild. An OK alternative to go along with their dry food.

